In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords, words
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
import os
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier


In [ ]:
data = pd.read_csv('/content/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df = data.copy()

In [ ]:
df.shape

(50000, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
# Checking null values
df.isna().sum()

,0
review,0
sentiment,0


In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
#one hot encoding
# label encoder
df.replace({'sentiment':{'positive':1, "negative":0}}, inplace = True )
df.head()

<ipython-input-21-2f73f8975606>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'sentiment':{'positive':1, "negative":0}}, inplace = True )


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [ ]:
#LSTM -> LONG SHORT TREM MEMORY
#RUN -> TEXTUAL DATA


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_data.shape, test_data.shape

((40000, 2), (10000, 2))

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])

In [ ]:
x_train=pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen = 200)
x_test=pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen = 200)

In [ ]:
x_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [ ]:
x_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [ ]:
y_train=train_data['sentiment']
y_test=test_data['sentiment']

In [ ]:
y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


In [ ]:
#LSTM MODEL BULDING

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_shape=(200,)))  # Use input_shape instead of input_length
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 197s 380ms/step - accuracy: 0.7276 - loss: 0.5233 - val_accuracy: 0.7760 - val_loss: 0.4801
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 198s 380ms/step - accuracy: 0.8416 - loss: 0.3774 - val_accuracy: 0.8549 - val_loss: 0.3403
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 381ms/step - accuracy: 0.8845 - loss: 0.2924 - val_accuracy: 0.8674 - val_loss: 0.3204
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 381ms/step - accuracy: 0.8953 - loss: 0.2663 - val_accuracy: 0.8652 - val_loss: 0.3220
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 199s 375ms/step - accuracy: 0.9085 - loss: 0.2296 - val_accuracy: 0.8700 - val_loss: 0.3251


In [ ]:
model.save("model.h5")

In [ ]:
import joblib
joblib.dump(Tokenizer, 'tokenizer.pkl')

['tokenizer.pkl']

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 111ms/step - accuracy: 0.8714 - loss: 0.3041


In [ ]:
print(loss)
print(accuracy)

0.3069741427898407
0.8715999722480774


In [ ]:
# Bulding predective system

In [ ]:
del predective_system

In [ ]:
def predective_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequences = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequences)
  sentement = "positive" if prediction[0][0]>0.5 else "negative"
  return sentement

In [71]:
predective_system("This move was fantasic")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step


'positive'

In [ ]:
predective_system("This move was not bad")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step


'positive'

In [73]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.4 MB/s eta 0:00:00


In [74]:
import gradio as gr
title = "Movie Reviews Sentiment Analysis"


app = gr.Interface(predective_system, inputs="textbox", outputs="textbox", title=title)
app.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a142cef93c703bfcbf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
